# install libraries

In [ ]:
#instll this library - read the doc 
!pip install fuzzywuzzy

In [ ]:
!pip install python-Levenshtein # install this to make fuzzy matching faster

# Import libraries

In [ ]:
#Import pandas
import pandas as pd

#Impor regex
import re

#Import csv
import csv

from IPython.utils.text import strip_email_quotes

from fuzzywuzzy import process, fuzz

# Choose folder to work on 

In [ ]:
#Insert the folder you are working on (e.g. folder = '18 Living with Social Media') and the VCS number (e.g. vcs_number = '18')

folder = '21 Achieving Peak Performance'
vcs_number = '21'

### set up working directory

In [ ]:
##Connecting with Gdrive
from google.colab import drive
drive.mount('/content/drive')

#Import Module
import os
  
# Folder Path
path = f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}'

# Change the directory
os.chdir(path)

# Question - answers from users 

##Clean .txt files  

In [ ]:
#Get a list of txt files inside the folder :
def file_list():
  global file_path_list
  file_path_list=[] 
  ## iterate through all files
  for file in os.listdir():
      # Check whether file is in text format or not
      if file.endswith('.txt'):
          file_path = f"{path}/{file}"
          file_path_list.append(file_path)

In [ ]:
def ignore_files_cleaned():
    global log_files_cleaned
    log_files_cleaned=[]# set to empty
    global file_path_list # set this variable to global 
    file_path = 'log_files_cleaned.csv'

    try:
        # Load the list from the CSV file
        with open(file_path, 'r') as csvfile:
            reader = csv.reader(csvfile)
            log_files_cleaned = list(reader)[0]

    #if the file is not found        
    except FileNotFoundError:
        print(f"File '{file_path}' not found.THIS MIGHT BE THE FIRST TIME FILES ARE CLEANED\n ALL .txt FILES WILL BE LOADED")

    # remove elements from file_path_list that were already cleaned
    file_path_list = [x for x in file_path_list if x not in log_files_cleaned]

In [ ]:
## FUNCTION opens a file, iterates over each line in the file, removes the newline character from each line, and appends the modified lines to a list.

def generate_script(file_path): 
  global script
  script=[] ## set script to empty
  with open(file_path, 'r') as f:
    for line in f:
      # remove newline character
      clean_line=line.rstrip('\n')
      #check if line is empty
      if clean_line !='':
         # append to list 
        script.append(clean_line)  

In [ ]:
#write log_files_cleaned to csv 
def write_list(l):
  file_path = 'log_files_cleaned.csv'  # Path to the output CSV file
  
   # Save the list to a CSV file
  with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(l)

### Search for Question-Answer line with regex  

In [ ]:

#Searching for questions & answers rows:
#If you print script you will notice that the rows with the questions, will be preceded by a ### 2022/07/05 06:45:00 | 18 Living with Social Media (Complete) ###; 
# and the answers row will be always be preceded by a ### 2022/07/05 06:44:58 | Visitor #189139665 ### hence the regular expression. 

def extract_q_a():
  global regexq
  global regexa
  global q_a
  global q_a_clean
  global first_match
  q_a=[] ## set q_a to empty 
  q_a_clean = [] ## set to empty 
  regexq = re.compile(r'\W{3}\s\d{4}\W\d{2}\W\d{2}\s\d{2}\W\d{2}\W\d{2}\s\W\s\d')  #Regular expression to signal a question row.
  regexa = re.compile(r'\W{3}\s\d{4}\W\d{2}\W\d{2}\s\d{2}\W\d{2}\W\d{2}\s\W\sVisitor') #Regular expression to signal a answer row.

  ### WE WILL ONLY START COUNTING AFTER THE FIRST QUESTION. TO AVOID THOSE LINES  WHEN A CHAT STARTS SUCH AS ('hello holi')

  first_match = False  # Flag to track the first match of regexq

  for index, row in enumerate(script):
      if not first_match and re.match(regexq, row):
          first_match = True  # Set the flag to True after the first match

      #if the line is a question :
      if first_match and re.match(regexq, row):
          q_a.append(row[4:23].strip()) # remove extra word and non word character
          q_a.append(script[index+1].strip())#The question or the answer, will be always next of the row with the regex.

      # if the line is an answer:
      if first_match and  re.match(regexa, row):
          q_a.append(row[35:45].strip()) # remove extra word and non word character
          q_a.append(script[index+1].strip()) #The question or the answer, will be always next of the row with the regex.
          
          # if the text of the row is an answer,we want to retrieve both the question and the answer:
          # generate list of lists [date,question,user,answer] 
          q_a_clean.append(q_a[-4:])  #We are creating a list of list with date, question, user number and answer)


### Iterating over all the txt files in the folder 

In [ ]:
file_list() # returns a list of files (file_path_list)
ignore_files_cleaned() # returns (file_path_list) after removing the paths that were inside log_of_files_cleaned. if it exists

#file_path_list #check the list returned

df_q_a=pd.DataFrame()

#loop over the list of files 

for file_path in (file_path_list):
  generate_script(file_path) # returns all the conversation (script)
  #add path to the log
  log_files_cleaned.append(file_path) 

  extract_q_a() # returns questions and answers  (q_a) and q_a_clean
  
  #write q_a_clean to a dataframe 
  # Create a temporary DataFrame for the current iteration's list
  temp_df = pd.DataFrame(q_a_clean, columns =['created_at', 'question','user_number','answer'])

  # concat the temporary DataFrame to the main DataFrame
  df_q_a = pd.concat([df_q_a,temp_df], ignore_index=True)


#write the list of file_path to the log in .csv
write_list(log_files_cleaned)

## CHECKPOINT questions-answers

### Save  Cleaned questions-answers  DataFrame

In [ ]:
# save dataframe (UNCOMENT THIS FOR SAVING )

#df_q_a.to_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_test_nahuel.csv')

### Read saved  questions-answers DataFrame

In [ ]:
# read saved dataFrame 

# Specify the file path
file_path = (f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_test_nahuel.csv')

# Read the CSV file into a DataFrame
df_q_a = pd.read_csv(file_path,index_col=0)

#remove trailing '.' from questions and andswers 
df_q_a['question']=df_q_a['question'].str.rstrip('. !')
df_q_a['answer']=df_q_a['answer'].str.rstrip('. !')

In [ ]:
# we can use a list of questions to filter by 

In [ ]:
question_list=[]

# Users data ( email-user number) 

make sure there is a .csv file in data processed with the name:

VCS{vcs_number}_users_db.csv

columns: @email , @id

### Read users data 

In [ ]:
#Importing user's info from Drive
users = pd.read_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/VCS{vcs_number}_users_db.csv')

users.rename(columns = {'@email':'user_email', '@id':'user_number'}, inplace = True)

users['user_number']=users['user_number'].astype(int)

##Join questions-answer to user data 

In [ ]:

# set user_number as string 
#df_q_a['user_number'] = df_q_a['user_number'].str.replace(r'[^0-9]', '',regex=True) ##uncoment or coment this as needed 
df_q_a['user_number'] = df_q_a['user_number'].astype(int)

In [ ]:
#Merging both dataframes
all_data = df_q_a.merge(users, on = 'user_number')

In [ ]:
all_data.sort_values('created_at',ascending=False)

# Question - Catalogue 

make sure there is a .csv file in data processed with the name:

question_catalogue

columns: question_v1 , answers_v1 ,	type , reverse_answer? [yes/no]

* in the column answers in the excel file . REPLACE a) b) c) etc with 1,2,3 etc .

* fill in wich health tracking questions are reverse or not: **If 1 is good > reverse = yes** 




## Read question_catalogue file 

In [ ]:
# read question_catalogue
# Specify the file path
file_path = (f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/question_catalogue.csv')

# Read the CSV file into a DataFrame
question_catalogue = pd.read_csv(file_path)

In [ ]:
question_catalogue.head(5)

### Clean question_catalogue

In [ ]:
#split answer column by \n
question_catalogue['answers_v1'] = question_catalogue['answers_v1'].apply(lambda x: x.split('\n'))

#remove blank and empty strings 
question_catalogue['answers_v1'] = question_catalogue['answers_v1'].apply(lambda x: [item.strip() for item in x if item.strip()])

# separate answers into different rows 
question_catalogue=question_catalogue.explode('answers_v1')

# separate number from text into different columns 
question_catalogue[['answer_number', 'answer_text']] = question_catalogue['answers_v1'].str.extract(r'(\d+)(.*)')

#remove trailing '.' from  answers 
question_catalogue['answer_text']=question_catalogue['answer_text'].str.strip('. !')

#remove trailing \r from questions 
question_catalogue['question_v1']=question_catalogue['question_v1'].str.rstrip('\r')

#change data type of the answer_number column 
question_catalogue['answer_number'] = question_catalogue['answer_number'].astype(int)

#change data type of the reverse_answer column 
question_catalogue.loc[:, 'reverse_answer? [yes/no]'] = question_catalogue['reverse_answer? [yes/no]'].astype(str)

#remove trailing spaces 

question_catalogue.loc[:, 'reverse_answer? [yes/no]'] = question_catalogue['reverse_answer? [yes/no]'].str.strip(' ')

In [ ]:
#check if there are null values, meaning the regex is wrong 
null_values = question_catalogue['answer_number'].isnull().sum() 
null_values

In [ ]:
# change name of question_v1 column
question_catalogue['question']=question_catalogue['question_v1']

# create another dataframe with only the columns we care about 
question_catalogue_clean=question_catalogue[['question',
                                            'answer_number',
                                            'answer_text',
                                            'type',
                                            'reverse_answer? [yes/no]']]
question_catalogue_clean.reset_index(inplace=True)

## Health tracking answer values 

In [ ]:
## SELECT HEALTH TRACKING AND BUILD THE LOGIC TO ASSING A VALUE DEPENDING ON THE ANSWER NUMBER,AND REVERSE SCALE AND THE SCALE OF 0 TO 10000

In [ ]:
filter=question_catalogue_clean['type']=='Health Tracking'

question_catalogue_clean[filter][['question','answer_number']].groupby('question').count()

## here we see that all health tracking questions have 5 answers 
## every answer goes from 1 to 5 
## if reverse=false 5=10000 and 1=0
## if reverse=true 1=10000 and 5=0

In [ ]:
#only health tracking has values in the reverse_answer column, there is no need to filter by type of question

for index, row in question_catalogue_clean.iterrows():

  reverse=question_catalogue_clean.loc[index,'reverse_answer? [yes/no]']

  if reverse=='no':
    question_catalogue_clean.loc[index,'answer_value']=(question_catalogue_clean.loc[index,'answer_number']-1)*2500
  if reverse=='yes':
    question_catalogue_clean.loc[index,'answer_value']=10000-(question_catalogue_clean.loc[index,'answer_number']-1)*2500


## if reverse=no 5=10000 and 1=0
## if reverse=yes 1=10000 and 5=0

if reverse is false that means that for the answer score, 5 = 10000; 4=7500; 3=5000; 2=2500 and 1 = 0. If the reverse is true the scale change direction, so 1=10000 and start decreasing in steps of 2500 until 5=0.


**If 1 is good > reverse = yes**




In [ ]:
# check answer values 

question_catalogue_clean[filter]

## Health literacy answer Values 

In [ ]:
filter=question_catalogue_clean['type']=='Health Literacy'

question_catalogue_clean[filter][['question','answer_number']].groupby('question').count()

## here we see that health literacy have a variable number of answer numbers 
## answer values can go from 0 to 1 . 1 being positive 
## there is no consistency in answer number and answer value. we will have to evaluate that case by case 

In [ ]:
# manually add the answer_value for all health literacy questions:
filter=question_catalogue_clean['type']=='Health Literacy'

# do you want to reset the health_literacy values?
valid_inputs = ['Y', 'N','n','y']
user_input = input('do you want to reset all answer_values from health_literacy ?\n[Y/N]')

while user_input not in valid_inputs:
    print("Invalid input. Please enter Y or N.")
    user_input = input('do you want to reset all answer_values from health_literacy ?\n[Y/N]')

if user_input=='Y' or user_input=='y':
  question_catalogue_clean.loc[filter, 'answer_value'] = None

# Loop through each row in the DataFrame
print('input 0 or 1 for this questions-answers')
for index, row in question_catalogue_clean[filter].iterrows():
    #if there is already a value for answer, go next 
    if pd.isnull(question_catalogue_clean.loc[index, 'answer_value']):
      # Display the prompt with the value from the 'answer' column
      user_input = input(row['question']+'\n'+row['answer_text'] + ": ")
    
       # Update the 'value' column in the DataFrame
      question_catalogue_clean.loc[index, 'answer_value'] = user_input
    else:
      continue
print('All answers already have a value !')

In [ ]:
# check values 
question_catalogue_clean[filter].head()

## General Questions answer values 

In [ ]:
## all general questions should be 0 

In [ ]:
filter=question_catalogue_clean['type']=='General'

question_catalogue_clean.loc[filter,'answer_value']=0

## CHECKPOINT 

### Save cleaned question_catalogue

In [ ]:
#check entire dataframe of question_catalogue 
question_catalogue_clean.head()

In [ ]:
# save dataframe (UNCOMENT THIS TO SAVE)
## question_catalogue_clean.to_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}question_catalogue_clean.csv')

### Read saved question_catalogue_clean

In [ ]:
# read saved question_catalogue_clean dataFrame 

# Specify the file path
file_path = (f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}question_catalogue_clean.csv')

# Read the CSV file into a DataFrame
question_catalogue_clean = pd.read_csv(file_path,index_col=0)

# Merge similar Questions using matching scores 


## questions from users 

#### Check similarity of questions inside logs




In [ ]:
#check that all questions from the all_data dataframe are unique using fuzzy matching

unique_questions=all_data[['question','answer']].groupby(['question']).count().sort_values('answer',ascending=False)
unique_questions.rename(columns={'answer': 'answer_num'}, inplace=True)
unique_questions.reset_index(inplace=True)

In [ ]:
unique_questions

In [ ]:
# Function to find similar string based on fuzzy matching and return similarity score
def find_similar_string(string, choices):
    choices = choices.drop(choices[choices == string].index)  # Exclude the string being evaluated
    best_match = process.extractOne(string, choices, scorer=fuzz.token_sort_ratio)
    if best_match is None or best_match[0] == string:
        return "", 0  # Return empty string and similarity score of 0
    else:
        return best_match[0], best_match[1]  # Return best match string and similarity score

# Apply fuzzy matching to create new columns for similar question, similarity score, and corresponding answer
result = unique_questions['question'].apply(lambda x: find_similar_string(x, unique_questions['question']))
unique_questions[['similar_question', 'similarity_score']] = pd.DataFrame(result.values.tolist(), index=result.index)

# Add corresponding answer column based on similar_question
unique_questions['similar_question_answer_num'] = unique_questions['similar_question'].map(unique_questions.set_index('question')['answer_num'])

In [ ]:
#add percentage of lost answers 
unique_questions['perc_dif_answers']=unique_questions['similar_question_answer_num']/unique_questions['answer_num']

#filter by similarity of the questions 
filter_similarity=unique_questions['similarity_score']>70

#filter by number of answers in the similar question
filter_lost_answers=unique_questions['similar_question_answer_num']>10

#filter by percentage of answers in the similar question
filter_lost_perc=unique_questions['perc_dif_answers']>0.1

#de-duplicate question - similar_question pairs 
filter_duplicate=unique_questions['answer_num']>unique_questions['similar_question_answer_num']


unique_questions[filter_similarity&filter_lost_answers&filter_lost_perc&filter_duplicate].sort_values('similar_question_answer_num',ascending=False)

##### make dict with replacements
If no replacements are needed leave the dictionary empty 


In [ ]:
# Format:
"""
replacements:{'similar_question':'question',
              'similar_question':'question'}
"""
replacements={''}

In [ ]:
## create a new dataframe for the cleaned data 
all_data_cleaned=all_data

## replace the questions using the dict :
all_data_cleaned['question']=all_data['question'].replace(replacements)


#### check similarity with questions in the catalogue 

In [ ]:
## make dataframe 
# Get unique questions from all_data_cleaned
unique_questions_all_data = pd.DataFrame({'question': all_data_cleaned['question'].unique()})

# Get unique questions from question_catalogue_clean
unique_questions_question_catalogue = pd.DataFrame({'question': question_catalogue_clean['question'].unique()})

In [ ]:
# Function to find the most approximate match from df2 for a given question
def find_approximate_match(question, df2_questions):
    exact_matches = [q for q in df2_questions if q != question]  # Exclude exact matches
    best_match = process.extractOne(question, exact_matches, scorer=fuzz.token_sort_ratio)
    return best_match[0], best_match[1]

# Load the two DataFrames
df1 = unique_questions_all_data  # DataFrame 1 with questions
df2 = unique_questions_question_catalogue # DataFrame 2 with questions

# Get a list of questions and matching scores from df2
df2_questions = df2['question'].tolist()

# Apply fuzzy matching to find the most approximate match from df2 for each question in df1
df1[['approximate_match', 'similarity_score']] = df1['question'].apply(lambda x: pd.Series(find_approximate_match(x, df2_questions)))

# Create a new DataFrame with questions from df1, their most approximate matches from df2, and matching scores
result_df = pd.DataFrame({'question_df1': df1['question'], 'approximate_match_df2': df1['approximate_match'], 'similarity_score': df1['similarity_score']})


In [ ]:
result_df

In [ ]:
#filter by similarity of the questions 
filter_similarity=(result_df['similarity_score']>82)


pd.set_option('display.max_colwidth', 150)

result_df[filter_similarity].sort_values('similarity_score',ascending=False)


##### make the replacements
The questions in the dataframe above are the ones that will be replaced.

**Change filter_similarity until the dataframe only has the questions we want replaced**


In [ ]:
# select replacements :
replacements_df=result_df[filter_similarity].sort_values('similarity_score',ascending=False)

# Convert DataFrame to dictionary
replacements = dict(zip(replacements_df['question_df1'], replacements_df['approximate_match_df2']))

## replace the questions using the dict :
all_data_cleaned['question']=all_data['question'].replace(replacements)

In [ ]:
#check dataframe 
all_data_cleaned.head()

cleaned dataframe for all_data 

In [ ]:
#check the dataframe 
all_data_cleaned

## Answers from users 

#### check similarity of answers inside logs 

In [ ]:
#check that all answers from the all_data_cleaned dataframe are unique using fuzzy matching

unique_answers=all_data_cleaned[['question','answer']].groupby(['answer']).count().sort_values('question',ascending=False)
unique_answers.rename(columns={'question': 'question_num'}, inplace=True)
unique_answers.reset_index(inplace=True)

In [ ]:
unique_answers

In [ ]:
# Function to find similar string based on fuzzy matching and return similarity score
def find_similar_string(string, choices):
    choices = choices.drop(choices[choices == string].index)  # Exclude the string being evaluated
    best_match = process.extractOne(string, choices, scorer=fuzz.token_sort_ratio)
    if best_match is None or best_match[0] == string:
        return "", 0  # Return empty string and similarity score of 0
    else:
        return best_match[0], best_match[1]  # Return best match string and similarity score

# Apply fuzzy matching to create new columns for similar question, similarity score, and corresponding answer
result = unique_answers['answer'].apply(lambda x: find_similar_string(x, unique_answers['answer']))
unique_answers[['similar_answer', 'similarity_score']] = pd.DataFrame(result.values.tolist(), index=result.index)

# Add corresponding answer column based on similar_question
unique_answers['similar_answer_question_num'] = unique_answers['similar_answer'].map(unique_answers.set_index('answer')['question_num'])

In [ ]:

#add percentage of lost questions
unique_answers['perc_dif_questions']=unique_answers['similar_answer_question_num']/unique_answers['question_num']

#filter by similarity of the questions 
filter_similarity=unique_answers['similarity_score']>80

#filter by number of answers in the similar question
filter_lost_answers=unique_answers['similar_answer_question_num']>10

#filter by percentage of answers in the similar question
filter_lost_perc=unique_answers['perc_dif_questions']>0.1

#de-duplicate question - similar_question pairs 
filter_duplicate=unique_answers['question_num']>unique_answers['similar_answer_question_num']


unique_answers[filter_similarity&filter_lost_answers&filter_lost_perc&filter_duplicate].sort_values('similar_answer_question_num',ascending=False)

##### **make a dict with the replacements**
If no replacements are needed leave the dictionary empty 


In [ ]:
# Format:
"""
replacements:{'similar_answer':'answer',
              'similar_answer':'answer'}
"""
replacements={'':''}

In [ ]:
## replace the questions using the dict :
all_data_cleaned['question']=all_data['question'].replace(replacements)

In [ ]:
#check dataframe 
all_data_cleaned.head()

#Join dataframes: all_data to question_catalogue 

## Filter all_data 

In [ ]:
#only keep questions from the log that are in the question catalogue 
filter=all_data_cleaned['question'].isin(question_catalogue_clean['question'].tolist())
all_data_cleaned=all_data_cleaned[filter]

## Finding best match for answers 



In [ ]:
#This block of code works like this :

# For each row in the all_data_cleaned dataframe : ( wich is where we have the question logs and users data )
# Select the question and the answer given by the user
# Filters the question_catalogue to only show the answers for that question
# Search for an exact match between the answer given by the user and the posible answers for that question in the question_catalogue
# If a match is found : it writes it to the 'best_match' column 
# if a match is not found : we search for the best possible match in that subset  of posible answers to the question
# then  write the best match, together with its matching score  to 'best_match' and 'match_score'column

# at the end we need to filter the dataframe to save only high matching scores and exact matches.

# Create columns 
all_data_cleaned['best_match'] = None
all_data_cleaned['match_score'] = None

# Loop through each row in the DataFrame
for index, row in all_data_cleaned.iterrows():

    # Function to find the closest match
    def find_closest_match(query, choices):
        best_match = process.extractOne(query, choices, scorer=fuzz.token_sort_ratio)
        return best_match

    # Select the question value
    question = all_data_cleaned.loc[index, 'question']

    # Select the answer value
    answer = all_data_cleaned.loc[index, 'answer']

    # Create filter for the question catalogue
    filter = question_catalogue_clean['question'] == question

    # Filter the question catalogue
    question_catalogue_filtered = question_catalogue_clean[filter]

    # Perform the exact match search
    for index_q, row_q in question_catalogue_filtered.iterrows():
      if answer==question_catalogue_filtered.loc[index_q,'answer_text']:
        all_data_cleaned.loc[index, 'best_match'] = question_catalogue_filtered.loc[index_q, 'answer_text']
        break  # Exit the loop if a match is found

    # Check if any exact matches were found
    if all_data_cleaned.loc[index, 'best_match']==None:

      # Select the best match
      best_match = find_closest_match(answer, question_catalogue_filtered['answer_text'].tolist())

      # Write best match and matching score to DataFrame
      all_data_cleaned.loc[index, 'best_match'] = best_match[0]
      all_data_cleaned.loc[index, 'match_score'] = best_match[1]

all_data_cleaned

###check that the matches were done correctly 

In [ ]:
df=all_data_cleaned[['question','answer','best_match','match_score']]
unique_df = df.drop_duplicates(subset=['question', 'answer', 'best_match','match_score']).sort_values('match_score')
unique_df.head(20)

## Setting Matching scores threshold 

**Change filter_match_score until only the answers we want replaced are left in the filtered dataframe**

In [ ]:
## match score filter 
filter_match_score=all_data_cleaned['match_score']<=65 ##CHANGE THIS FILTER AND CHECK THE RESULTING DATAFRAME 


#check the dataframe 
all_data_cleaned[~filter_match_score]


### Apply filter to dataframe 

In [ ]:
#filter dataframe 
all_data_cleaned=all_data_cleaned[~filter_match_score].copy()

## join dataframes 

In [ ]:
## join dataframes

# Join the DataFrames using two columns as the join keys
merged_df = pd.merge(all_data_cleaned,
                     question_catalogue_clean, 
                     left_on=['question','best_match'],
                     right_on=['question','answer_text'],
                     how='inner')

In [ ]:
final_dataframe=merged_df[['created_at',
                          'question',
                          'user_number',
                          'answer',
                          'user_mail',
                          'answer_value']]

final_dataframe

#Save final dataframe 

#### overwrite
* Its the first time runing the script
* We want to make the dataframe again
when the data here is of all the log files (we process the entire folder) and 

In [ ]:
""" DELETE THIS ROW IF YOU WANT TO OVERRWRITE 

final_dataframe.to_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv')

""" ## DELETE THIS ROW IF YOU WANT TO OVERRWRITE 

#### Append 
* should be used when we want to add data partially ( for example we already have a data frame made with this script  and we get new data from landbot)


In [ ]:
""" ## DELETE THIS ROW IF YOU WANT TO APPEND


# Check if the CSV file exists
if os.path.isfile(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv'):
    # Read the existing CSV file into a dataframe
    existing_dataframe = pd.read_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv')
else:
    # Print an error message and exit the script
    print(f"File not found , is this the first time running this script for this folder ?")
    exit()

# Append the final_dataframe to the existing dataframe
appended_dataframe = existing_dataframe.append(final_dataframe)

# Write the appended dataframe to the CSV file
appended_dataframe.to_csv(f'/content/drive/My Drive/ SV9001 Landbot data extraction automation/{folder}/data_processed/vcs{vcs_number}_final.csv')

""" ## DELETE THIS ROW IF YOU WANT TO APPEND